In [1]:
%%time

import polars as pl
import sy

FILE_PATH = "/tmp/dbg1request.parquet"

df = (
  pl
  .scan_parquet(FILE_PATH, use_pyarrow=True, null_values="NULL")
  .collect()
)

df.select(["message_protocol", "cmci", "http_request", "hmci"])

CPU times: user 2.9 s, sys: 3.03 s, total: 5.94 s
Wall time: 948 ms


message_protocol,cmci,http_request,hmci
str,i32,bool,i32
"""coap""",1,null,-99
"""coap""",1,null,-99
"""http""",-1,true,1
"""http""",-1,false,-99
"""coap""",3,null,-99
"""coap""",3,null,-99
"""coap""",1,null,-99
"""coap""",1,null,-99
"""http""",-1,true,1


In [2]:
(
  df
  .groupby("message_protocol")
  .agg(
    pl.col("message_protocol").count().alias("count")
  )
)

message_protocol,count
str,u32
"""coap""",1719429
"""http""",62673


In [3]:
http_df = (
  df
  .filter(
    pl.col("message_protocol") == "http"
  )
)

http_request_df = (
  df
  .filter(
    (pl.col("http_request") == True)
    & (pl.col("http_request_method") == "get")
  )
)

http_response_df = (
  df
  .filter(
    (pl.col("http_request") == False)
    & (pl.col("http_response_code") == 200)
  )
)

print(f"{len(http_df)=}")
print(f"{len(http_request_df)=}")
print(f"{len(http_response_df)=}")

assert len(http_df) == len(http_request_df) + len(http_response_df)

http_df.select(["http_request", "hmci"])

len(http_df)=62673
len(http_request_df)=31139
len(http_response_df)=31534


http_request,hmci
bool,i32
true,1
false,-99
true,1
true,1
false,-99
false,-99
true,1
false,-99
true,1


In [4]:
(
  df
  .groupby("hmci")
  .agg(
    pl.col("hmci").count().alias("count")
  )
)

hmci,count
i32,u32
-99,1750963
1,31139
